## 关于这个笔记本

该笔记本由 B220400 班某咸鱼创建，用于自动化处理“波尔共振仪研究受迫振动”实验的数据。

数据请填入 `free_vibration.csv`，`damped_vibration.csv` 和 `forced_vibration.csv`，并将变量 `DAMPED_VIBRATION_DATA_10T` 修改为你自己的数据。

In [ ]:
FREE_VIBRATION_DATA_FILE = 'free_vibration.csv'
DAMPED_VIBRATION_DATA_FILE = 'damped_vibration.csv'
DAMPED_VIBRATION_DATA_10T = 15.740
FORCED_VIBRATION_DATA_FILE = 'forced_vibration.csv'

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Free Vibration 自由振动

In [ ]:
free_vibration_data = pd.read_csv(FREE_VIBRATION_DATA_FILE)

X = np.array(free_vibration_data[r'$T_0$'])[:, np.newaxis]
y = np.array(free_vibration_data[r'$\theta$'])

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)

prediction_line_label = rf'$\theta={float(model.coef_)}T_0+{float(model.intercept_)}$'
free_vibration_plot = sns.regplot(data=free_vibration_data, x=r'$T_0$', y=r'$\theta$', line_kws={'label': prediction_line_label}, ci=None)
free_vibration_plot.set_title(r'Free Vibration')
free_vibration_plot.set_xlabel(r'$T_0/s$')
free_vibration_plot.set_ylabel(r'$\theta/(\degree)$')
free_vibration_plot.legend()

## Forced Vibration 受迫振动

In [ ]:
forced_vibration_data = pd.read_csv(FORCED_VIBRATION_DATA_FILE)

from_theta_to_t0_model = LinearRegression()
from_theta_to_t0_model.fit(np.array(free_vibration_data[r'$\theta$'])[:, np.newaxis], np.array(free_vibration_data[r'$T_0$']))

forced_vibration_data[r'$T_0$']=from_theta_to_t0_model.predict(np.array(forced_vibration_data[r'$\theta$'])[:, np.newaxis])
forced_vibration_data[r'$\frac{\omega}{\omega_0}$']=forced_vibration_data[r'$T_0$'] / forced_vibration_data[r'$T$']

forced_vibration_data

In [ ]:
sns.lmplot(data=forced_vibration_data, x=r'$\frac{\omega}{\omega_0}$', y=r'$\theta$', order=4, ci=None)

In [ ]:
sns.lmplot(data=forced_vibration_data, x=r'$\frac{\omega}{\omega_0}$', y=r'$\varphi$', order=3, ci=None)

## Damped Vibration 阻尼振动

In [ ]:
damped_vibration_data = pd.read_csv(DAMPED_VIBRATION_DATA_FILE)
damped_vibration_data_T = DAMPED_VIBRATION_DATA_10T / 10
damped_vibration_data[r'$\frac{\theta_i}{\theta_{i+5}}$'] = damped_vibration_data[r'$\theta_i$'] / damped_vibration_data[r'$\theta_{i+5}$']
from math import log
damped_vibration_data[r'$\beta$'] = damped_vibration_data[r'$\frac{\theta_i}{\theta_{i+5}}$'].map(log) / 5 / damped_vibration_data_T

damped_vibration_data

In [ ]:
damped_vibration_data_beta = damped_vibration_data[r'$\beta$'].mean()
print(f'Beta 的平均值 = {damped_vibration_data_beta}')